In [1]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_csv('genshintwitternext.csv')
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')

content = df['content'].to_list()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [3]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

2020-11-09 00:08:53,988 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.197913  0.065183       1        1  8.931052
13    -0.315015 -0.116347       2        1  7.412908
14     0.164169 -0.058804       3        1  5.694754
4      0.077380  0.119007       4        1  5.344661
12     0.037730  0.077589       5        1  5.204110
5      0.087577  0.102817       6        1  4.991665
2      0.109182  0.037811       7        1  4.925199
7     -0.004328  0.179065       8        1  4.902970
18    -0.002010  0.166140       9        1  4.846515
17    -0.061010  0.073662      10        1  4.721841
1      0.150580 -0.094007      11        1  4.564233
0     -0.069440  0.051979      12        1  4.501331
15     0.025059  0.015799      13        1  4.467197
19    -0.112183 -0.059334      14        1  4.431540
16     0.097640 -0.022969      15        1  4.341646
9     -0.204057 -0.099623      16        1  4.243982
11     0.084781 -0.275658      17        1  4.141243
6     -0.054593 -0.064680      18        1  4.118096
8      0.052160  0.099664      19        1  4.114665
3      0.134290 -0.197294      20        1  4.100392, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1787.000000  1787.000000  Default  30.0000  30.0000
555          impact  1229.000000  1229.000000  Default  29.0000  29.0000
443   genshinimpact   506.000000   506.000000  Default  28.0000  28.0000
622            like   545.000000   545.000000  Default  27.0000  27.0000
173          childe   477.000000   477.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
467            good    36.357911   310.584668  Topic20  -4.1674   1.0490
87           beidou    24.290669   111.232592  Topic20  -4.5708   1.6726
286           diluc    36.683125   379.979597  Topic20  -4.1585   0.8563
1238        zhongli    32.533283   372.639479  Topic20  -4.2786   0.7557
862           razor    24.450342   157.860368  Topic20  -4.5642   1.3290

[975 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         9  0.783650          able
0        12  0.069146          able
0        14  0.138291          able
2         9  0.988040         abyss
4         7  0.079527  accidentally
...     ...       ...           ...
1238      7  0.888258       zhongli
1238     15  0.024152       zhongli
1238     20  0.088557       zhongli
1239     11  0.847562        zombie
1239     16  0.094174        zombie

[2295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 14, 15, 5, 13, 6, 3, 8, 19, 18, 2, 1, 16, 20, 17, 10, 12, 7, 9, 4])

In [4]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.234026  0.035259       1        1  9.218913
4      0.101381 -0.039548       2        1  5.460724
7     -0.099220  0.004587       3        1  5.266149
14     0.132780 -0.021930       4        1  5.199071
2      0.157134 -0.004763       5        1  5.175122
6     -0.082783 -0.121495       6        1  5.147352
19    -0.085772 -0.051640       7        1  5.103675
5      0.099088 -0.145581       8        1  5.044480
11     0.040860  0.223183       9        1  5.044265
12     0.014835  0.009191      10        1  4.825342
15     0.000818 -0.173745      11        1  4.758662
9     -0.130690  0.019770      12        1  4.756534
16     0.095508 -0.122198      13        1  4.644314
1      0.144565  0.137248      14        1  4.614136
17    -0.061368  0.024001      15        1  4.507312
18    -0.006000  0.052040      16        1  4.461387
8      0.132989  0.066430      17        1  4.330169
0      0.015177  0.057818      18        1  4.264774
13    -0.120555  0.071753      19        1  4.099906
3     -0.114719 -0.020379      20        1  4.077714, topic_info=               Term        Freq       Total Category  logprob  loglift
636            live  177.000000  177.000000  Default  30.0000  30.0000
443   genshinimpact  169.000000  169.000000  Default  29.0000  29.0000
765          paimon   97.000000   97.000000  Default  28.0000  28.0000
173          childe  167.000000  167.000000  Default  27.0000  27.0000
709            mona   91.000000   91.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
442         genshin   18.100453  430.941773  Topic20  -3.9389   0.0296
584            just   13.356868  192.019502  Topic20  -4.2428   0.5341
555          impact   14.733592  334.813400  Topic20  -4.1447   0.0762
431            game   11.291505  154.067093  Topic20  -4.4107   0.5863
1139         update    8.948324   50.395085  Topic20  -4.6433   1.4712

[978 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         7  0.882214  absolutely
1         8  0.088221  absolutely
2        10  0.042367       abyss
2        16  0.932078       abyss
3         5  0.854678         acc
...     ...       ...         ...
1238     13  0.049724     zhongli
1238     14  0.058011     zhongli
1238     16  0.008287     zhongli
1238     17  0.033149     zhongli
1239      7  0.831530      zombie

[2530 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 5, 8, 15, 3, 7, 20, 6, 12, 13, 16, 10, 17, 2, 18, 19, 9, 1, 14, 4])

In [5]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.438761 -0.103322       1        1  8.931052
13     0.459684  0.146516       2        1  7.412908
14    -0.255603  0.014229       3        1  5.694754
4     -0.206687 -0.156895       4        1  5.344661
12     0.125463 -0.386225       5        1  5.204110
5     -0.018440 -0.312537       6        1  4.991665
2     -0.324814 -0.298927       7        1  4.925199
7     -0.017055 -0.027128       8        1  4.902970
18     0.066283  0.157998       9        1  4.846515
17     0.188446 -0.150192      10        1  4.721841
1      0.302112 -0.340056      11        1  4.564233
0     -0.012430  0.381417      12        1  4.501331
15     0.288557  0.303025      13        1  4.467197
19    -0.296005  0.276867      14        1  4.431540
16     0.148035  0.402896      15        1  4.341646
9     -0.163412  0.419835      16        1  4.243982
11    -0.469862 -0.078023      17        1  4.141243
6     -0.402795  0.173932      18        1  4.118096
8      0.289014  0.033068      19        1  4.114665
3     -0.139252 -0.456478      20        1  4.100392, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1787.000000  1787.000000  Default  30.0000  30.0000
555          impact  1229.000000  1229.000000  Default  29.0000  29.0000
443   genshinimpact   506.000000   506.000000  Default  28.0000  28.0000
622            like   545.000000   545.000000  Default  27.0000  27.0000
173          childe   477.000000   477.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
467            good    36.357911   310.584668  Topic20  -4.1674   1.0490
87           beidou    24.290669   111.232592  Topic20  -4.5708   1.6726
286           diluc    36.683125   379.979597  Topic20  -4.1585   0.8563
1238        zhongli    32.533283   372.639479  Topic20  -4.2786   0.7557
862           razor    24.450342   157.860368  Topic20  -4.5642   1.3290

[975 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         9  0.783650          able
0        12  0.069146          able
0        14  0.138291          able
2         9  0.988040         abyss
4         7  0.079527  accidentally
...     ...       ...           ...
1238      7  0.888258       zhongli
1238     15  0.024152       zhongli
1238     20  0.088557       zhongli
1239     11  0.847562        zombie
1239     16  0.094174        zombie

[2295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 14, 15, 5, 13, 6, 3, 8, 19, 18, 2, 1, 16, 20, 17, 10, 12, 7, 9, 4])

In [6]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
10     -10.878068   10.538728       1        1  8.931052
13     -60.884026  -75.591820       2        1  7.412908
14      -2.336119  -53.289074       3        1  5.694754
4      -61.385319  -12.914634       4        1  5.344661
12     -28.737743  126.296997       5        1  5.204110
5        1.066606 -123.552811       6        1  4.991665
2       45.140720   -7.947869       7        1  4.925199
7     -140.795975  -98.530548       8        1  4.902970
18      61.531681  -83.240562       9        1  4.846515
17     -72.563057 -140.396881      10        1  4.721841
1       15.984491   63.578186      11        1  4.564233
0     -160.851624   68.945145      12        1  4.501331
15      85.961830   51.922935      13        1  4.467197
19     -99.020836  109.936348      14        1  4.431540
16    -176.764130   -8.681684      15        1  4.341646
9     -117.189880  -37.420200      16        1  4.243982
11     108.829918  -22.946203      17        1  4.141243
6       47.588097  121.934814      18        1  4.118096
8     -106.467834   30.176889      19        1  4.114665
3      -50.613804   59.328842      20        1  4.100392, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1787.000000  1787.000000  Default  30.0000  30.0000
555          impact  1229.000000  1229.000000  Default  29.0000  29.0000
443   genshinimpact   506.000000   506.000000  Default  28.0000  28.0000
622            like   545.000000   545.000000  Default  27.0000  27.0000
173          childe   477.000000   477.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
467            good    36.357911   310.584668  Topic20  -4.1674   1.0490
87           beidou    24.290669   111.232592  Topic20  -4.5708   1.6726
286           diluc    36.683125   379.979597  Topic20  -4.1585   0.8563
1238        zhongli    32.533283   372.639479  Topic20  -4.2786   0.7557
862           razor    24.450342   157.860368  Topic20  -4.5642   1.3290

[975 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         9  0.783650          able
0        12  0.069146          able
0        14  0.138291          able
2         9  0.988040         abyss
4         7  0.079527  accidentally
...     ...       ...           ...
1238      7  0.888258       zhongli
1238     15  0.024152       zhongli
1238     20  0.088557       zhongli
1239     11  0.847562        zombie
1239     16  0.094174        zombie

[2295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 14, 15, 5, 13, 6, 3, 8, 19, 18, 2, 1, 16, 20, 17, 10, 12, 7, 9, 4])